In [1]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# # from jupyterthemes import get_themes
# # import jupyterthemes as jt
# # from jupyterthemes.stylefx import set_nb_theme
# # set_nb_theme('solarizedd')

In [2]:
import requests
import pandas as pd

In [3]:
#download csv files for example:
def download_csv_files(years, path):
    
      for year in years:
            for half in ['Jan_Jun', 'Jul_Dec']:
            
                path = path

                save_path = f'{path}EIA930_SUBREGION_{year}_{half}.csv'

                # Define the URLs for the CSV files
                url = f'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_SUBREGION_{year}_{half}.csv'

                response = requests.get(url)

                response.raise_for_status()

                with open (save_path, 'wb') as file:
                    file.write(response.content)

                print(f'Successfully downloaded files to {save_path}')

        
        

In [4]:
path = r'D:/Users/ARH/gh-repos-n/python-sql-example/data/'
download_csv_files([2019,2020,2021], path)

Successfully downloaded files to D:/Users/ARH/gh-repos-n/python-sql-example/data/EIA930_SUBREGION_2019_Jan_Jun.csv
Successfully downloaded files to D:/Users/ARH/gh-repos-n/python-sql-example/data/EIA930_SUBREGION_2019_Jul_Dec.csv
Successfully downloaded files to D:/Users/ARH/gh-repos-n/python-sql-example/data/EIA930_SUBREGION_2020_Jan_Jun.csv
Successfully downloaded files to D:/Users/ARH/gh-repos-n/python-sql-example/data/EIA930_SUBREGION_2020_Jul_Dec.csv
Successfully downloaded files to D:/Users/ARH/gh-repos-n/python-sql-example/data/EIA930_SUBREGION_2021_Jan_Jun.csv
Successfully downloaded files to D:/Users/ARH/gh-repos-n/python-sql-example/data/EIA930_SUBREGION_2021_Jul_Dec.csv


In [5]:
df_example = pd.read_csv(r'D:/Users/ARH/gh-repos-n/python-sql-example/data/EIA930_SUBREGION_2019_Jan_Jun.csv')
df_example

,Balancing Authority,Data Date,Hour Number,Sub-Region,Demand (MW),Local Time at End of Hour,UTC Time at End of Hour
0,CISO,01/01/2019,1,PGAE,"9,851",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
1,CISO,01/01/2019,1,SCE,"9,932",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
2,CISO,01/01/2019,1,SDGE,"2,090",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
3,CISO,01/01/2019,1,VEA,92,01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
4,CISO,01/01/2019,2,PGAE,"9,514",01/01/2019 2:00:00 AM,01/01/2019 10:00:00 AM
...,...,...,...,...,...,...,...
356135,SWPP,06/30/2019,24,SPRM,422,07/01/2019 12:00:00 AM,07/01/2019 5:00:00 AM
356136,SWPP,06/30/2019,24,SPS,"4,425",07/01/2019 12:00:00 AM,07/01/2019 5:00:00 AM
356137,SWPP,06/30/2019,24,WAUE,"3,248",07/01/2019 12:00:00 AM,07/01/2019 5:00:00 AM
356138,SWPP,06/30/2019,24,WFEC,"1,081",07/01/2019 12:00:00 AM,07/01/2019 5:00:00 AM


# Working with SQL in Python

### Objectives:
    
#### SQLite - SQL syntax
    - Creating a simple database using pandas `to_sql`
    - Running custom sql code to query database
    - Using jupysql
    
#### Object-Relational Mappers (ORM) - pythonic way to create and interact with sqlite database
    - Creating database
    - Basic/advanced queries
    

# SQLite - SQL syntax

To start out, I'll demonstrate a simple method to read multiple CSV files from a folder, concatenate them, and write the combined data to a SQLite database. 

## Working with CSV Files and SQLite in Python

If you have multiple CSV files in a single folder and you want to combine them into a single SQLite database, you can use the following function.

In [6]:
import pandas as pd
import sqlite3
import glob
import os

def csv_to_sqlite(folder_path, database_file):
    # Create a SQLite connection
    conn = sqlite3.connect(database_file)
    
    # Build the file path pattern
    pattern = os.path.join(folder_path, '*.csv')

    # Find all CSV files in the folder
    csv_files = glob.glob(pattern)

    # Read and concatenate all found CSV files
    concatenated_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

    # Write the DataFrame to SQLite database
    concatenated_df.to_sql('example_data', conn, if_exists='replace', index=False)

    # Close the SQLite connection
    conn.close()



In [7]:
# Example usage
folder_path = r'D:/Users/ARH/gh-repos-n/python-sql-example/data/'
database_file = 'basic_database.db'
csv_to_sqlite(folder_path, database_file)

In [8]:
import sqlite3

#connect to your database
conn = sqlite3.connect('basic_database.db')

#create a cursor object using the cursor method
cur = conn.cursor()

#execute a query
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

#fetch the results
tables = cur.fetchall()

#print the results
for table in tables:
    print(table[0])

example_data


In [9]:
import pandas as pd
import sqlite3
from time import time
import datetime
from pathlib import Path


def _parse_sql(sql: str) -> str:
    """Properly interprets sql as file or query then processes"""

    if ".sql" == str(sql)[-4:]:
        if Path(sql).is_file():
            open_query = open(sql, "r")
            sql = open_query.read()
        else:
            raise ValueError(
                f"File not found at {sql}. "
                "If not a file, make sure your script doesn't end in .sql"
            )

    # pd.read_sql cannot handle semi colons
    sql = sql.replace(";", "")

    return sql

def sql_run(connection, sql, lower_cols=False, verbose=True) -> pd.DataFrame:
    """
    :description: Use db connection created to query
                  database with a single .sql file or string containing sql
                  query. Can handle comments and ';' character.

    :param connection: db connection 
    :type connection: cx_Oracle.Connection; sqlite
    :param sql: .sql file or string containing query you want to run
    :type sql: str
    :param lower_cols: convert column names to lowercase, by default False
    :type lower_cols: bool
    :param verbose: print sql run time, by default True
    :type verbose: bool

    :returns:  pd.DataFrame of queried data
    """
    t0 = time()
    df = pd.read_sql(_parse_sql(sql), connection)
    t1 = time()

    if lower_cols:
        df.columns = df.columns.str.lower()

    # Print query run time
    if verbose:
        print(str(datetime.timedelta(seconds=round(t1 - t0))))
        
    return df




### Use `sql_run` functions to query all tables names in database file

In [10]:

# Establish a database connection (example with SQLite)
conn = sqlite3.connect('basic_database.db')

# Write your SQL query as a string
sql_query = """
SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;
"""

# Run the query
result_df = sql_run(conn, sql_query)

# Display the results
print(result_df)

0:00:00
           name
0  example_data


### Use `sql_run` functions to write custom sql in notebook cell

In [11]:

# Establish a database connection (example with SQLite)
conn = sqlite3.connect('basic_database.db')

# Write your SQL query as a string
sql_query = """
SELECT *
FROM example_data;
"""

# Run the query
result_df = sql_run(conn, sql_query)

# Display the results
result_df


0:00:08


,Balancing Authority,Data Date,Hour Number,Sub-Region,Demand (MW),Local Time at End of Hour,UTC Time at End of Hour
0,CISO,01/01/2019,1,PGAE,"9,851",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
1,CISO,01/01/2019,1,SCE,"9,932",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
2,CISO,01/01/2019,1,SDGE,"2,090",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
3,CISO,01/01/2019,1,VEA,92,01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
4,CISO,01/01/2019,2,PGAE,"9,514",01/01/2019 2:00:00 AM,01/01/2019 10:00:00 AM
...,...,...,...,...,...,...,...
2156923,SWPP,12/31/2021,24,SPRM,270,01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM
2156924,SWPP,12/31/2021,24,SPS,"3,136",01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM
2156925,SWPP,12/31/2021,24,WAUE,"4,538",01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM
2156926,SWPP,12/31/2021,24,WFEC,946,01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM


### Use `sql_run` functions to write custom sql in notebook cell cont. -- MISO, sub-region 0001

In [12]:
# Establish a database connection (example with SQLite)
conn = sqlite3.connect('basic_database.db')

# Write your SQL query as a string
sql_query = """
SELECT *
FROM example_data
WHERE "Balancing Authority"= 'MISO' AND "Sub-Region" = '0001';
"""

# Run the query
result_df = sql_run(conn, sql_query)

# Display the results
result_df

0:00:01


,Balancing Authority,Data Date,Hour Number,Sub-Region,Demand (MW),Local Time at End of Hour,UTC Time at End of Hour
0,MISO,01/01/2019,1,0001,"11,293",01/01/2019 1:00:00 AM,01/01/2019 6:00:00 AM
1,MISO,01/01/2019,2,0001,"11,049",01/01/2019 2:00:00 AM,01/01/2019 7:00:00 AM
2,MISO,01/01/2019,3,0001,"10,791",01/01/2019 3:00:00 AM,01/01/2019 8:00:00 AM
3,MISO,01/01/2019,4,0001,"10,610",01/01/2019 4:00:00 AM,01/01/2019 9:00:00 AM
4,MISO,01/01/2019,5,0001,"10,529",01/01/2019 5:00:00 AM,01/01/2019 10:00:00 AM
...,...,...,...,...,...,...,...
26299,MISO,12/31/2021,20,0001,"12,849",12/31/2021 8:00:00 PM,01/01/2022 1:00:00 AM
26300,MISO,12/31/2021,21,0001,"12,583",12/31/2021 9:00:00 PM,01/01/2022 2:00:00 AM
26301,MISO,12/31/2021,22,0001,"12,322",12/31/2021 10:00:00 PM,01/01/2022 3:00:00 AM
26302,MISO,12/31/2021,23,0001,"12,101",12/31/2021 11:00:00 PM,01/01/2022 4:00:00 AM


### Use `sql_run` functions to write custom sql in *.sql* file

In [13]:

# Establish a database connection (example with SQLite)
conn = sqlite3.connect('basic_database.db')

# Path to your .sql file
sql_file_path = "sql/query_miso_0001.sql"

# Run the query from the .sql file
result_df = sql_run(conn, sql_file_path)

# Display the results
result_df


0:00:01


,Balancing Authority,Data Date,Hour Number,Sub-Region,Demand (MW),Local Time at End of Hour,UTC Time at End of Hour
0,MISO,01/01/2019,1,0001,"11,293",01/01/2019 1:00:00 AM,01/01/2019 6:00:00 AM
1,MISO,01/01/2019,2,0001,"11,049",01/01/2019 2:00:00 AM,01/01/2019 7:00:00 AM
2,MISO,01/01/2019,3,0001,"10,791",01/01/2019 3:00:00 AM,01/01/2019 8:00:00 AM
3,MISO,01/01/2019,4,0001,"10,610",01/01/2019 4:00:00 AM,01/01/2019 9:00:00 AM
4,MISO,01/01/2019,5,0001,"10,529",01/01/2019 5:00:00 AM,01/01/2019 10:00:00 AM
...,...,...,...,...,...,...,...
26299,MISO,12/31/2021,20,0001,"12,849",12/31/2021 8:00:00 PM,01/01/2022 1:00:00 AM
26300,MISO,12/31/2021,21,0001,"12,583",12/31/2021 9:00:00 PM,01/01/2022 2:00:00 AM
26301,MISO,12/31/2021,22,0001,"12,322",12/31/2021 10:00:00 PM,01/01/2022 3:00:00 AM
26302,MISO,12/31/2021,23,0001,"12,101",12/31/2021 11:00:00 PM,01/01/2022 4:00:00 AM


## ORM

In [14]:
import os
import glob
import pandas as pd

def concat_all(folder_path):
    all_data = glob.glob(os.path.join(folder_path, "*.csv"))
    df_all = []
    for filename in all_data:
        df = pd.read_csv(filename, index_col=None, header=0)
        df_all.append(df)
    concat_data = pd.concat(df_all, axis=0, ignore_index=True)
    return concat_data

#read in all csv data into a single pandas dataframe
df = concat_all('data/')
df
    
    
        

,Balancing Authority,Data Date,Hour Number,Sub-Region,Demand (MW),Local Time at End of Hour,UTC Time at End of Hour
0,CISO,01/01/2019,1,PGAE,"9,851",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
1,CISO,01/01/2019,1,SCE,"9,932",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
2,CISO,01/01/2019,1,SDGE,"2,090",01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
3,CISO,01/01/2019,1,VEA,92,01/01/2019 1:00:00 AM,01/01/2019 9:00:00 AM
4,CISO,01/01/2019,2,PGAE,"9,514",01/01/2019 2:00:00 AM,01/01/2019 10:00:00 AM
...,...,...,...,...,...,...,...
2156923,SWPP,12/31/2021,24,SPRM,270,01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM
2156924,SWPP,12/31/2021,24,SPS,"3,136",01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM
2156925,SWPP,12/31/2021,24,WAUE,"4,538",01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM
2156926,SWPP,12/31/2021,24,WFEC,946,01/01/2022 12:00:00 AM,01/01/2022 6:00:00 AM


In [15]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, DateTime, Integer
from sqlalchemy.orm import sessionmaker, declarative_base
from time import time
import datetime


# Define the base class for ORM
Base = declarative_base()

#to create SQLite db, you need to create an engine that represents the connection. Then use it to create all tables
#defined by your model
engine = create_engine('sqlite:///ORM_database.db')
Base.metadata.create_all(engine)


class EIA930Subregion(Base):
    __tablename__ = 'eia930_subregion_raw'
    id = Column(String, primary_key=True)
    balancing_authority = Column(String)
    sub_region = Column(String)
    demand = Column(Integer)
    utc_time_at_end_of_hour = Column(DateTime)
    hour = Column(Integer)
    date = Column(String)
    local_time = Column(DateTime)
    # Add other columns as needed


# create_database(engine)

desired_columns = {
        'Balancing Authority': 'balancing_authority',
        'UTC Time at End of Hour': 'utc_time_at_end_of_hour',
        'Sub-Region': 'sub_region',
        'Demand (MW)': 'demand',
        'Hour Number': 'hour',
        'Data Date': 'date',
        'Local Time at End of Hour': 'local_time'
        # Add other desired columns here
}

 # Convert relevant columns to numeric
cols_to_convert = ['UTC Time at End of Hour', 'Local Time at End of Hour']
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_datetime, errors='coerce')

df.rename(columns=desired_columns, inplace=True)
df['id'] = df['sub_region'] + '_' + df['utc_time_at_end_of_hour'].astype(str)
#remove commas and nans in demand data
df['demand'] = df['demand'].replace(',', '', regex=True)
df['demand'] = pd.to_numeric(df['demand'], errors='coerce').fillna(0).astype(int)
df

,balancing_authority,date,hour,sub_region,demand,local_time,utc_time_at_end_of_hour,id
0,CISO,01/01/2019,1,PGAE,9851,2019-01-01 01:00:00,2019-01-01 09:00:00,PGAE_2019-01-01 09:00:00
1,CISO,01/01/2019,1,SCE,9932,2019-01-01 01:00:00,2019-01-01 09:00:00,SCE_2019-01-01 09:00:00
2,CISO,01/01/2019,1,SDGE,2090,2019-01-01 01:00:00,2019-01-01 09:00:00,SDGE_2019-01-01 09:00:00
3,CISO,01/01/2019,1,VEA,92,2019-01-01 01:00:00,2019-01-01 09:00:00,VEA_2019-01-01 09:00:00
4,CISO,01/01/2019,2,PGAE,9514,2019-01-01 02:00:00,2019-01-01 10:00:00,PGAE_2019-01-01 10:00:00
...,...,...,...,...,...,...,...,...
2156923,SWPP,12/31/2021,24,SPRM,270,2022-01-01 00:00:00,2022-01-01 06:00:00,SPRM_2022-01-01 06:00:00
2156924,SWPP,12/31/2021,24,SPS,3136,2022-01-01 00:00:00,2022-01-01 06:00:00,SPS_2022-01-01 06:00:00
2156925,SWPP,12/31/2021,24,WAUE,4538,2022-01-01 00:00:00,2022-01-01 06:00:00,WAUE_2022-01-01 06:00:00
2156926,SWPP,12/31/2021,24,WFEC,946,2022-01-01 00:00:00,2022-01-01 06:00:00,WFEC_2022-01-01 06:00:00


In [17]:
engine = create_engine('sqlite:///ORM_database.db')
Base.metadata.create_all(engine)

#a session is a way to interact with your database. It allows you to query and manipulate data
Session = sessionmaker(bind=engine)
session = Session()

#bulk insert data from dataframe
session.bulk_insert_mappings(EIA930Subregion, df.to_dict(orient='records'))
session.commit()


In [18]:
#query all the data using ORM syntax -- sql equivalance to select*
query = session.query(EIA930Subregion).statement
df = pd.read_sql(query, session.bind)
df

,id,balancing_authority,sub_region,demand,utc_time_at_end_of_hour,hour,date,local_time
0,PGAE_2019-01-01 09:00:00,CISO,PGAE,9851,2019-01-01 09:00:00,1,01/01/2019,2019-01-01 01:00:00
1,SCE_2019-01-01 09:00:00,CISO,SCE,9932,2019-01-01 09:00:00,1,01/01/2019,2019-01-01 01:00:00
2,SDGE_2019-01-01 09:00:00,CISO,SDGE,2090,2019-01-01 09:00:00,1,01/01/2019,2019-01-01 01:00:00
3,VEA_2019-01-01 09:00:00,CISO,VEA,92,2019-01-01 09:00:00,1,01/01/2019,2019-01-01 01:00:00
4,PGAE_2019-01-01 10:00:00,CISO,PGAE,9514,2019-01-01 10:00:00,2,01/01/2019,2019-01-01 02:00:00
...,...,...,...,...,...,...,...,...
2156923,SPRM_2022-01-01 06:00:00,SWPP,SPRM,270,2022-01-01 06:00:00,24,12/31/2021,2022-01-01 00:00:00
2156924,SPS_2022-01-01 06:00:00,SWPP,SPS,3136,2022-01-01 06:00:00,24,12/31/2021,2022-01-01 00:00:00
2156925,WAUE_2022-01-01 06:00:00,SWPP,WAUE,4538,2022-01-01 06:00:00,24,12/31/2021,2022-01-01 00:00:00
2156926,WFEC_2022-01-01 06:00:00,SWPP,WFEC,946,2022-01-01 06:00:00,24,12/31/2021,2022-01-01 00:00:00


In [19]:
from sqlalchemy import and_
query = session.query(EIA930Subregion.id, EIA930Subregion.sub_region, EIA930Subregion.__table__.c['balancing_authority'], EIA930Subregion.__table__.c['demand']).filter(and_(EIA930Subregion.__table__.c['balancing_authority'] == 'MISO', EIA930Subregion.sub_region == '0001' )).statement
df = pd.read_sql(query, session.bind)
df


,id,sub_region,balancing_authority,demand
0,0001_2019-01-01 06:00:00,0001,MISO,11293
1,0001_2019-01-01 07:00:00,0001,MISO,11049
2,0001_2019-01-01 08:00:00,0001,MISO,10791
3,0001_2019-01-01 09:00:00,0001,MISO,10610
4,0001_2019-01-01 10:00:00,0001,MISO,10529
...,...,...,...,...
26299,0001_2022-01-01 01:00:00,0001,MISO,12849
26300,0001_2022-01-01 02:00:00,0001,MISO,12583
26301,0001_2022-01-01 03:00:00,0001,MISO,12322
26302,0001_2022-01-01 04:00:00,0001,MISO,12101
